#  Chapter 3:  Technical Indicators

In [8]:
# import dependencies

import pandas as pd
import datetime as dt
import numpy as np
from typing import List, Dict, Tuple, Any, Callable

from pypm import indicators

In [3]:
# import and load data...
# loading SPY DF...

df = pd.read_csv('data/SPY.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index(df['date'], inplace=True)
df.drop(columns=['date'], inplace=True)

In [4]:
df.head()

,open,close,low,high,volume
date,,,,,
2010-01-04,112.37,113.33,111.51,113.39,118944600
2010-01-05,113.26,113.63,112.85,113.68,111579900
2010-01-06,113.52,113.71,113.43,113.99,116074400
2010-01-07,113.50,114.19,113.18,114.33,131091100
2010-01-08,113.89,114.57,113.66,114.62,126402800


## 3.1  Rolling Functions and Algorithms

### 3.1.1  Simple Moving Average

In [5]:
# calculating simple moving average (SMA) in pandas

def calculate_simple_moving_average(series: pd.Series, n: int=20) -> pd.Series:
    '''
    Calculates the simple moving average
    '''
    return series.rolling(n).mean()


In [10]:
print(calculate_simple_moving_average(df['close']))

date
2010-01-04         NaN
2010-01-05         NaN
2010-01-06         NaN
2010-01-07         NaN
2010-01-08         NaN
                ...   
2019-12-24    316.1215
2019-12-26    316.5645
2019-12-27    316.9335
2019-12-30    317.2720
2019-12-31    317.7830
Name: close, Length: 2516, dtype: float64


### Simple Moving Stdev

In [18]:
def calculate_simple_moving_sample_stdev(series: pd.Series, n: int=20) -> pd.Series:
    """Calculates the simple moving average"""
    return series.rolling(n).std()

### 3.1.2  Efficient Windowing Functions

In [14]:
# 3.2  calculating the simple moving average inefficiently, using pure python
def slow_moving_average(values: List[float], m: int=20):
    '''
    This is 0(nm) time, because it re-computes the sum at every step
    1 + 2 + 3 + 4 ... / m
    2 + 3 + 4 + 5 ... / m
    3 + 4 + 5 + 6 ... / m
    4 + 5 + 6 + 7 ... / m
    and so on ...
    Leading to approx (m-1) * n individual additions
    '''
    
    # Initial values
    moving_average = [None] * (m-1)
    
    for i in range(m-1, len(values)):
        the_average = np.mean(values[(i-m+1):i+1])
        moving_average.append(the_average)
        
    return moving_average

In [15]:
# 3.3  calculating the simple moving average efficently
def fast_moving_average(values: List[float], m: int=20):
    '''
    This is 0(n) time, because it keeps track of the intermediate sum.
    Loading to approx 2n individual additions.
    '''
    
    # Initial values
    moving_average = [None] * (m-1)
    accumulator = sum(values[:m])
    moving_average.append(accumulator / m)
    
    for i in range(m, len(values)):
        accumulator -= values[i-m]
        accumulator += values[i]
        moving_average.append(accumulator / m)
        
    return moving_average

## 3.2  Oscillators

### 3.2.1  Moving Average Convergence Divergence Oscillator (MACD)

In [16]:
# 3.4  calculating the MACD Oscillator

def calculate_macd_oscillator(series: pd.Series, n1: int=5, n2: int=34) -> pd.Series:
    '''
    Calculate the moving average convergence divergence oscillator, given a
    short moving average length n1 and a long moving average length n2.
    '''
    assert n1 < n2, f'n1 must be less than n2'
    return calculate_simple_moving_average(series, n1) - \
        calculate_simple_moving_average(series, n2)

## 3.3  Overlays

### 3.3.1  Bollinger Bands

In [17]:
# 3.5  calculate Bollinger Bands

def calculate_bollinger_bands(series: pd.Series, n: int=20) -> pd.DataFrame:
    '''
    calculate the Bollinger Bands and returns them as a dataframe
    '''
    
    sma = calculate_simple_moving_average(series, n)
    stdev = calculate_simple_moving_stdev(series, n)
    
    return pd.DataFrame({
        'middle': sma,
        'upper': sma + 2 * stdev,
        'lower': sma - 2 * stdev
    })

## 3.4  Volume-based Indicators

### 3.4.1  Chaikin Money Flow

In [19]:
# The Chaikin Money Flow (CMF) is an oscillator that depends on both volume and candlestick data. - see text for formula

# 3.6 calculating Chaikin Money Flow

def calculate_money_flow_volume_series(df: pd.DataFrame) -> pd.Series:
    '''
    Calculates money flow series
    '''
    mfv = df['volume'] * (2*df['close'] - df['high'] - df['low']) / (df['high'] - df['low'])
    
    return mfv

def calculate_money_flow_volume(df: pd.DataFrame, n: int=20) -> pd.Series:
    '''
    Calculates money flow volume, or q_t in our formula
    '''
    return calculate_money_flow_volume_series(df).rolling(n).sum()

def calculate_chaikin_money_flow(df: pd.DataFrame, n: int=20) -> pd.Series:
    '''
    Calculates the Chaikin Money Flow
    '''
    return calculate_money_flow_volume(df, n) / df['volume'].rolling(n).sum()

## 3.5  Signals

### 3.5.2  Generating Signals from Indicators

In [22]:
# 3.7  Creating signals from indicators

def create_macd_signal(series: pd.Series, n1: int=5, n2: int=34) -> pd.Series:
    '''
    Create a momentum-based signal based on the MACD crossover principle.
    Generate a buy signal whe the MACD cross above zero, and a sell signal when it crosses below zero.
    '''
    # Calculate the macd and get the signs of the values
    macd = calculate_macd_oscillator(series, n1, n2)
    macd_sign = np.sign(macd)
    
    # Create a copy shifted by some amount
    macd_shifted_sign = macd_sign.shift(1, axis=0)
    
    # Multiply the sign by the boolean. This will have the effect of casting
    # the boolean to an integer (either 0 or 1) and then multiple by the sign
    # (either -1, 0 or 1).
    return macd_sign * (macd_sign != macd_shifted_sign)

def create_bollinger_band_signal(series: pd.Series, n: int=20) -> pd.Series:
    '''
    Create a reversal-based signal based on the upper and lower bands of the 
    Bollinger bands. Generate a buy signal when the price is below the lower band,
    and a sell signal when the price is above the upper band
    '''
    bollinger_bands = calculate_bollinger_bands(series, n)
    sell = series > bollinger_bands['upper']
    buy = series < bollinger_bands['lower']
    return (1*buy - 1*sell)